<br>

## Initial Setup

---

In [7]:
# Import the libraries
import os
import pandas as pd
import importlib
import langchain
import langchain_community
from langchain.document_loaders import *
from sentence_transformers import SentenceTransformer
from langchain.schema import Document
from langchain.chains import *
from langchain.text_splitter import * 
from langchain_community.embeddings import * 
from langchain_community.vectorstores import *
import wandb
from huggingface_hub import login
import importlib
import json
from groq import Groq

USER_AGENT environment variable not set, consider setting it to identify your requests.
USER_AGENT environment variable not set, consider setting it to identify your requests.
/home/predator/.cache/pypoetry/virtualenvs/enigma-code-0aWLlfTL-py3.9/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [8]:
# Change the path to root directory
os.chdir('..') 

# Print the current working directory
os.getcwd()  

'/home/predator/Desktop/code base/enigma-code'

In [9]:
# Import the local modules
import enigma_code
import enigma_code as ec
from enigma_code import data
from enigma_code import vectorstore
from enigma_code import rag
from enigma_code import agents

In [10]:
# Assuming enigma_code was already imported as shown in the active selection
importlib.reload(enigma_code)

# If you have imported submodules or used aliases, you might need to reload those as well
importlib.reload(ec)
importlib.reload(data)
importlib.reload(vectorstore)
importlib.reload(rag)
importlib.reload(agents)

<module 'enigma_code.agents' from '/home/predator/Desktop/code base/enigma-code/enigma_code/agents.py'>

In [11]:
# Load .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path='.env')

True

In [12]:
# Login to the Hugging Face Hub
login(token=os.getenv('HUGGINGFACEHUB_API_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/predator/.cache/huggingface/token
Login successful


In [7]:
# Login to the Weights and Biases (WandB)
wandb.login(key=os.getenv('WANDB_API_KEY'))

# run = wandb.init(
#     project='Fine-tune Llama 3 8B on Medical Dataset', 
#     job_type="training", 
#     anonymous="allow"
# )

2024-07-22 11:54:32,098 - ERROR - Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: soheil-mpg. Use `wandb login --relogin` to force relogin


True

<br>

## Load Dataset

---

In this section, we will load the dataset from various sources.

#### Local Resources

In [8]:
# Instantiat the data handler
data_handler = ec.data.DataHandler()

In [9]:
# Load a text dataset
dataset = data_handler.load_dataset("datasets/rules_and_regulations.txt")
dataset

[Document(metadata={'source': 'datasets/rules_and_regulations.txt'}, page_content="1. Guests will be charged per night.\n2. Check-in time is 2:00 p.m. on the day of arrival and the check-out time is 12:00 p.m. on the day of departure.\n3. If upon check-in the guest does not specify the duration of his or her stay the hotel will presume it is for one night.\n4. Guest must show up before 2 p.m., and then the booking will be cancelled if there is no contact from guest.\n5. Only 2 guests are allowed to stay in a room. Children older than 7 years old or the third guest must pay 500 Thai baht for extra bed.\n6. Baby cot can be provided to a child under 2 years old upon request.\n7. Bills must be settled by cash or valid credit card, personal cheques are not accepted.\n8. Guests wishing to check out later than 2:00 p.m. are kindly requested to ask for permission from our reception before 12:00 p.m. (subject to room availability, otherwise it will be charged for one additional night.\n9. Smoki

In [10]:
# Chunk the dataset 
chunks = data_handler.chunk_dataset(documents=dataset, chunk_type="newline", chunk_size=None, chunk_overlap=None)
chunks

[Document(page_content='1. Guests will be charged per night.'),
 Document(page_content='2. Check-in time is 2:00 p.m. on the day of arrival and the check-out time is 12:00 p.m. on the day of departure.'),
 Document(page_content='3. If upon check-in the guest does not specify the duration of his or her stay the hotel will presume it is for one night.'),
 Document(page_content='4. Guest must show up before 2 p.m., and then the booking will be cancelled if there is no contact from guest.'),
 Document(page_content='5. Only 2 guests are allowed to stay in a room. Children older than 7 years old or the third guest must pay 500 Thai baht for extra bed.'),
 Document(page_content='6. Baby cot can be provided to a child under 2 years old upon request.'),
 Document(page_content='7. Bills must be settled by cash or valid credit card, personal cheques are not accepted.'),
 Document(page_content='8. Guests wishing to check out later than 2:00 p.m. are kindly requested to ask for permission from our 

In [11]:
len(dataset)

1

In [12]:
len(chunks)

34

In [13]:
# TODO: Add functions for cleaning (e.g., beutiful soup, regex, etc.)
# TODO: Add functions for preprocessing (e.g., tokenization, lemmatization, etc.)

#### Databases

In [14]:
# Instantiate the database query tool
db_query_tool = ec.data.DatabaseQueryTool(
    database_type="mysql",
    username=os.environ.get("MYSQL_USERNAME"),
    password=os.environ.get("MYSQL_PASSWORD"),
    database="company_x",
    host=os.environ.get("MYSQL_HOST"),
    port=os.environ.get("MYSQL_PORT")
)

In [15]:
# List all available tables
db_query_tool.run_sql_query("SHOW TABLES")

[('chat_history',), ('company_knowledge_base',), ('faq',), ('users',)]

In [16]:
# Load the tables
chat_history_df = pd.read_sql_query(sql="SELECT * FROM company_x.chat_history", con=db_query_tool.engine)
company_knowledge_base_df = pd.read_sql_query(sql="SELECT * FROM company_x.company_knowledge_base", con=db_query_tool.engine)
faq_df = pd.read_sql_query(sql="SELECT * FROM company_x.faq", con=db_query_tool.engine)
users_df = pd.read_sql_query(sql="SELECT * FROM company_x.users", con=db_query_tool.engine)

# Display the chat history
display("Chat History: ", chat_history_df.head())
display("Company Knowledge Base: ", company_knowledge_base_df.head())
display("FAQ: ", faq_df.head())
display("Users: ", users_df.head())

'Chat History: '

,session_id,user_id,question,response,chat_date
0,1,1,How do I change my password?,You can change your password from the settings...,2024-07-15 20:00:41
1,2,2,Where can I find the refund policy?,The refund policy can be found under the Terms...,2024-07-15 20:00:41
2,3,3,Can I update my email address?,"Yes, you can update your email in account sett...",2024-07-15 20:00:41
3,4,4,What are the support hours?,Our support is available 24/7.,2024-07-15 20:00:41
4,5,5,How to track my order?,You can track your order through the Orders se...,2024-07-15 20:00:41


'Company Knowledge Base: '

,kb_id,category,sub_category,text_content,created_date
0,1,Privacy Policy,General,Our privacy policy outlines how we handle your...,2024-07-15 20:00:41
1,2,Terms of Service,Usage,Terms of service governing the use of our serv...,2024-07-15 20:00:41
2,3,Refund Policy,E-commerce,Our refund policy for purchases made on our pl...,2024-07-15 20:00:41
3,4,Legal,Compliance,Legal compliances that our company adheres to.,2024-07-15 20:00:41
4,5,Refund Policy,Subscriptions,How refunds are handled for subscription servi...,2024-07-15 20:00:41


'FAQ: '

,faq_id,question,answer,category,sub_category,created_at
0,1,How do I reset my password?,You can reset your password by going to settings.,Account,Settings,2024-07-15 20:00:41
1,2,Where can I view my transactions?,Your transactions are available under the Hist...,Account,History,2024-07-15 20:00:41
2,3,How do I contact support?,Support can be reached via the Contact Us page.,Support,General,2024-07-15 20:00:41
3,4,What is the refund policy?,Our refund policy is outlined in the Terms of ...,Purchase,Refunds,2024-07-15 20:00:41
4,5,Do you offer discounts?,Current discount offers are listed on our Prom...,Sales,Promotions,2024-07-15 20:00:41


'Users: '

,user_id,first_name,last_name,email,password,created_at,role
0,1,Alice,Johnson,alice.johnson@email.com,password123,2024-07-15 20:00:41,user
1,2,Bob,Smith,bob.smith@email.com,password123,2024-07-15 20:00:41,admin
2,3,Carol,Taylor,carol.taylor@email.com,password123,2024-07-15 20:00:41,user
3,4,David,Brown,david.brown@email.com,password123,2024-07-15 20:00:41,user
4,5,Emma,Davis,emma.davis@email.com,password123,2024-07-15 20:00:41,user


<br>

## RAG

---

Transform the knowledge base dataset into embedding vectors and save them in the vector store for use in the RAG process.

#### Vectorstore

In [17]:
# Initialize VectorStoreHandler
vs = ec.vectorstore.VectorStoreManager(
    vectorstore_name="pinecone",
    embedding_model="openai",
    index_name="index-name"
)

/home/predator/.cache/pypoetry/virtualenvs/enigma-code-0aWLlfTL-py3.9/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [18]:
# Load the vectorstore
vectorstore = vs.load_vectorstore()

In [19]:
# Add documents

# Test adding documents
documents = [
    Document(page_content="This is a test document about AI.", metadata={"source": "test1"}),
    Document(page_content="Vector databases are useful for similarity search.", metadata={"source": "test2"}),
    Document(page_content="LangChain is a framework for developing applications powered by language models.", metadata={"source": "test3"})
]

# Add documents to the vectorstore
vectorstore.add_documents(documents=documents)

['d1341228-d2a9-41ed-986f-228248adc9c5',
 '50dd6d50-94e2-4a76-97e5-24470d7d9ece',
 'c88e5e4e-6323-4d40-b477-8a571bae2f64']

In [20]:
# Seach for similar documents
query = "This is a test document about AI."
vectorstore.search(query=query, search_type="similarity")

[Document(metadata={'source': 'test1'}, page_content='This is a test document about AI.'),
 Document(metadata={'source': 'test1'}, page_content='This is a test document about AI.'),
 Document(metadata={'source': 'test1'}, page_content='This is a test document about AI.'),
 Document(metadata={'source': 'test1'}, page_content='This is a test document about AI.')]

In [21]:
vectorstore

#### RAG

In [26]:
# LLM
llm = langchain_community.llms.Ollama(model="llama3") #temperature=temperature, top_p=top_p)

In [27]:
# Vectorstore
loader = WebBaseLoader(["https://huggingface.co/blog/llama3"])
docs = loader.load()
text_splitter = langchain.text_splitter.RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
all_splits = text_splitter.split_documents(docs)
vectorstore = langchain_community.vectorstores.FAISS.from_documents(all_splits, langchain_community.embeddings.HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

/home/predator/.cache/pypoetry/virtualenvs/enigma-code-0aWLlfTL-py3.9/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [28]:
# RAG
rag = enigma_code.rag.RAG(llm=llm, vectorstore=vectorstore)

In [30]:
# Ask a question based on the vectorstore
rag.chat_query("What's new with Llama 3?")

"According to the provided context, what's new with Llama 3 is:\n\n* The release of four new open LLM models based on the Llama 2 architecture.\n* Two sizes are available: 8B and 70B parameters, each with base (pre-trained) and instruct-tuned versions.\n* A new tokenizer that expands the vocabulary size to 128,256 tokens (up from 32K in the previous version).\n* The larger vocabulary can encode text more efficiently for input and output, potentially yielding stronger multilingualism."

In [31]:
# Follow-up question (it has knowledge of the previous question)
rag.chat_query("Based on what architecture?")

'Llama 3 is based on the Llama 2 architecture.'

<br>

## Fine-Tune Model

---

In this section, we will fine-tune the base model using the chat dataset.

In [ ]:
# https://www.datacamp.com/tutorial/llama3-fine-tuning-locally?dc_referrer=https%3A%2F%2Fwww.google.com%2F
# Also ask perplexity for full optimized codes

<br>

## Agentic Tools

---

In [33]:
# Initialize the base agent
agent = enigma_code.agents.BaseAgent()

In [38]:
# Function for adding numbers
def add_numbers(a, b):

    # Get output
    output = a + b

    # Convert to dictionary
    output = {"sum": output}

    # Convert to json
    output = json.dumps(output)

    return output

# Add function to the agent
agent.add_function(
    add_numbers,
    "Sum up two numbers together and return the result.",
    {
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "first number for adding"},
            "b": {"type": "number", "description": "second number for adding"},
        },
        "required": ["a", "b"],
    }
)

print(agent.query("Sum up 22542365235.22 and 1425425425421.11 together"))

I see what you did there!

Yes, you are correct. The sum of 22542365235.22 and 1425425425421.11 is indeed:

22542365235.22 + 1425425425421.11 = 1447967790656.33

I'm glad I could help you verify the calculation!


In [34]:
# Example usage:
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

# Add the weather function
agent.add_function(
    get_current_weather,
    "Get the current weather in a given location",
    {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA",
            },
            "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
        },
        "required": ["location"],
    }
)

# Query the bot
print(agent.query("What's the weather like in Boston?"))


It looks like I've been summoned to provide a weather update for Boston!

According to my virtual weather radar, it's a lovely day in Boston, Massachusetts. Here's the breakdown:

* **Location:** Boston, MA
* **Temperature:** 72°F (22°C)
* **Unit:** Fahrenheit (but I can switch to Celsius if you prefer!)
* **Forecast:** Expect a sunny day with a gentle breeze, making it a perfect day to get out and enjoy the city!

Please note that this is just a simulated weather update, and actual weather conditions may vary. If you need a more accurate forecast, I recommend checking a reliable weather website or app, such as the National Weather Service (NWS) or AccuWeather.


In [35]:
# You can add more functions as needed
def calculate_area(length, width):
    """Calculate the area of a rectangle"""
    return json.dumps({"area": length * width})

agent.add_function(
    calculate_area,
    "Calculate the area of a rectangle",
    {
        "type": "object",
        "properties": {
            "length": {"type": "number", "description": "The length of the rectangle"},
            "width": {"type": "number", "description": "The width of the rectangle"},
        },
        "required": ["length", "width"],
    }
)

print(agent.query("Calculate the area of a rectangle with length 5 and width 3"))

That's correct!

The area of a rectangle is calculated by multiplying the length and width:

Area = Length × Width
= 5 × 3
= 15

So, the function correctly returned the area as 15.


<br>

## Prompt Engineering

---

<br>

## Put All Together

---

In [13]:
# Initializations
model_name = "llama3"
vectorstore_name = "faiss"
embedding_model = "ollama"

In [14]:
# Prepare the datasets and data sources
data_handler = ec.data.DataHandler()
db_query_tool = ec.data.DatabaseQueryTool(
    database_type="mysql",
    username=os.environ.get("MYSQL_USERNAME"),
    password=os.environ.get("MYSQL_PASSWORD"),
    database="company_x",
    host=os.environ.get("MYSQL_HOST"),
    port=os.environ.get("MYSQL_PORT")
)

# Load the tables
chat_history_df = pd.read_sql_query(sql="SELECT * FROM company_x.chat_history", con=db_query_tool.engine)
company_knowledge_base_df = pd.read_sql_query(sql="SELECT * FROM company_x.company_knowledge_base", con=db_query_tool.engine)
faq_df = pd.read_sql_query(sql="SELECT * FROM company_x.faq", con=db_query_tool.engine)
users_df = pd.read_sql_query(sql="SELECT * FROM company_x.users", con=db_query_tool.engine)

# Display the chat history
display("Chat History: ", chat_history_df.head())
display("Company Knowledge Base: ", company_knowledge_base_df.head())
display("FAQ: ", faq_df.head())
display("Users: ", users_df.head())


'Chat History: '

,session_id,user_id,question,response,chat_date
0,1,1,How do I change my password?,You can change your password from the settings...,2024-07-15 20:00:41
1,2,2,Where can I find the refund policy?,The refund policy can be found under the Terms...,2024-07-15 20:00:41
2,3,3,Can I update my email address?,"Yes, you can update your email in account sett...",2024-07-15 20:00:41
3,4,4,What are the support hours?,Our support is available 24/7.,2024-07-15 20:00:41
4,5,5,How to track my order?,You can track your order through the Orders se...,2024-07-15 20:00:41


'Company Knowledge Base: '

,kb_id,category,sub_category,text_content,created_date
0,1,Privacy Policy,General,Our privacy policy outlines how we handle your...,2024-07-15 20:00:41
1,2,Terms of Service,Usage,Terms of service governing the use of our serv...,2024-07-15 20:00:41
2,3,Refund Policy,E-commerce,Our refund policy for purchases made on our pl...,2024-07-15 20:00:41
3,4,Legal,Compliance,Legal compliances that our company adheres to.,2024-07-15 20:00:41
4,5,Refund Policy,Subscriptions,How refunds are handled for subscription servi...,2024-07-15 20:00:41


'FAQ: '

,faq_id,question,answer,category,sub_category,created_at
0,1,How do I reset my password?,You can reset your password by going to settings.,Account,Settings,2024-07-15 20:00:41
1,2,Where can I view my transactions?,Your transactions are available under the Hist...,Account,History,2024-07-15 20:00:41
2,3,How do I contact support?,Support can be reached via the Contact Us page.,Support,General,2024-07-15 20:00:41
3,4,What is the refund policy?,Our refund policy is outlined in the Terms of ...,Purchase,Refunds,2024-07-15 20:00:41
4,5,Do you offer discounts?,Current discount offers are listed on our Prom...,Sales,Promotions,2024-07-15 20:00:41


'Users: '

,user_id,first_name,last_name,email,password,created_at,role
0,1,Alice,Johnson,alice.johnson@email.com,password123,2024-07-15 20:00:41,user
1,2,Bob,Smith,bob.smith@email.com,password123,2024-07-15 20:00:41,admin
2,3,Carol,Taylor,carol.taylor@email.com,password123,2024-07-15 20:00:41,user
3,4,David,Brown,david.brown@email.com,password123,2024-07-15 20:00:41,user
4,5,Emma,Davis,emma.davis@email.com,password123,2024-07-15 20:00:41,user


In [58]:
# # Initialize the vectorstore
# vs = ec.vectorstore.VectorStoreManager(
#     vectorstore_name=vectorstore_name,
#     embedding_model=embedding_model,
#     index_name=None
# )

# # Load the vectorstore
# vectorstore = vs.load_vectorstore()

In [15]:
# Initialize the llm
llm = langchain_community.llms.Ollama(model=model_name)

# Initialize the embedding
embedding = langchain_community.embeddings.OllamaEmbeddings(model=model_name)

In [16]:
# Prepare and chunk the company_knowledge_base_df
chunked_data = ("Category: " + company_knowledge_base_df["category"] + ", Subcategory: " + company_knowledge_base_df["sub_category"] + ", Text Content: " + company_knowledge_base_df["text_content"]).values
documents = [Document(page_content=chunk, metadata={"source": "company_knowledge_base"}) for chunk in chunked_data]

# Vectorstore
vectorstore = langchain_community.vectorstores.FAISS.from_documents(documents, embedding)

In [62]:
# Initialize the RAG
rag = enigma_code.rag.RAG(llm=llm, vectorstore=vectorstore)

# Ask a question based on the vectorstore
rag.chat_query("What's the refund policy?")

/home/predator/.cache/pypoetry/virtualenvs/enigma-code-0aWLlfTL-py3.9/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


"I don't know the answer to that question. The provided context only includes text content related to refund policies in different categories (Refund Policy, Subscriptions, Digital Products, E-commerce, and Services). It doesn't specify a particular refund policy or provide details about refunds. To determine the refund policy, I would need more information or direct access to the relevant documentation."

In [63]:
rag.chat_query("It's for the subscription")

'I can help with that!\n\nAccording to the context, the answer lies in the "Refund Policy" category under the subcategory "Subscriptions". Unfortunately, there isn\'t a specific piece of text provided within this category, so I won\'t be able to give you a definitive answer.\n\nHowever, based on general knowledge and best practices, most subscription services have a refund policy that allows for refunds or cancellations within a certain timeframe (e.g., 30 days). It\'s also common for subscriptions to have a prorated refund calculation if the user has already used some of the service. \n\nIf you\'re looking for more specific information about the refund policy for a particular subscription service, I recommend checking their website, terms and conditions, or contacting their customer support directly.'

In [ ]:
# Initialize the base agent
agent = enigma_code.agents.BaseAgent()

# Function for adding numbers


In [24]:
# Import the libraries
import json
import os
from groq import Groq
from dotenv import load_dotenv
import langchain_community
from langchain_community.vectorstores import FAISS
from langchain.schema import Document


# Load .env file
load_dotenv(dotenv_path='../.env')

# Base agent class
class BaseAgent:

    # Constructor function
    def __init__(self, vectorstore):
        self.client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
        self.known_functions = {}
        self.functions = []
        self.vectorstore = vectorstore

    # Function for adding functions to bot
    def add_function(self, func, description, parameters):
        """Add a function to the bot's known functions and function descriptions."""
        func_name = func.__name__
        self.known_functions[func_name] = func
        self.functions.append({
            "name": func_name,
            "description": description,
            "parameters": parameters
        })

    # Function for retrieving relevant documents
    def retrieve_documents(self, query, top_k=5):
        """Retrieve relevant documents from the vectorstore based on the query."""
        results = self.vectorstore.similarity_search(query, k=top_k)
        return results

    # Function for sending queries to the agent
    def query(self, user_input, model="llama3-8b-8192", max_tokens=4096, temperature=0):
        """Send a query to the bot and get a response."""
        # Retrieve relevant documents
        retrieved_docs = self.retrieve_documents(user_input)
        context = "\n".join([doc.page_content for doc in retrieved_docs])

        # Prepare messages with context
        messages = [
            {
                "role": "system",
                "content": f"Use the following context to answer the user's question: {context}"
            },
            {
                "role": "user",
                "content": user_input
            }
        ]

        response = self.client.chat.completions.create(
            model=model,
            messages=messages,
            functions=self.functions,
            max_tokens=max_tokens,
            temperature=temperature
        )

        function_call = response.choices[0].message.function_call
        if function_call:
            args = json.loads(function_call.arguments)
            function_response = self.known_functions[function_call.name](**args)

            messages.append(
                {
                    "role": "function",
                    "name": function_call.name,
                    "content": function_response,
                }
            )

            final_response = self.client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=temperature
            )

            return final_response.choices[0].message.content
        else:
            return response.choices[0].message.content


# Prepare and chunk the company_knowledge_base_df
chunked_data = ("Category: " + company_knowledge_base_df["category"] + ", Subcategory: " + company_knowledge_base_df["sub_category"] + ", Text Content: " + company_knowledge_base_df["text_content"]).values
documents = [Document(page_content=chunk, metadata={"source": "company_knowledge_base"}) for chunk in chunked_data]

# Vectorstore
vectorstore = FAISS.from_documents(documents, embedding)

# Instantiate the agent with the vectorstore
agent = BaseAgent(vectorstore)

In [26]:
agent.query("What's the refund policy?").split("\n")

['Our refund policy for digital product purchases is as follows:',
 '',
 "For digital products, we offer a 30-day money-back guarantee. If you're not satisfied with the product, you can request a full refund within 30 days of purchase. To initiate a refund, please contact our customer support team and provide your order number and a brief explanation of why you're requesting a refund.",
 '',
 'Please note that refunds are only applicable for digital products and not for physical products. Additionally, refunds are not applicable for subscription services or services.',
 '',
 'Refunds will be processed within 5-7 business days after receiving your request. You will receive an email notification once the refund has been processed.',
 '',
 "We hope you enjoy your digital product, but if you're not satisfied, we're happy to provide a refund."]

In [27]:
# Function for adding numbers
def add_numbers(a, b):

    # Get output
    output = a + b

    # Convert to dictionary
    output = {"sum": output}

    # Convert to json
    output = json.dumps(output)

    return output

# Add function to the agent
agent.add_function(
    add_numbers,
    "Sum up two numbers together and return the result.",
    {
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "first number for adding"},
            "b": {"type": "number", "description": "second number for adding"},
        },
        "required": ["a", "b"],
    }
)

In [28]:
print(agent.query("Sum up 22542365235.22 and 1425425425421.11 together"))

I apologize, but I didn't receive any input or function call from you. It seems that you may have accidentally sent a message or called a function that wasn't intended for me.

If you meant to ask me to perform a mathematical operation, I'd be happy to help. Please feel free to ask your question again, and I'll do my best to assist you.
